In [5]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

import warnings

warnings.filterwarnings('ignore')


# n개 이상의 데이터 리뷰를 남긴 유저, 맥주를 걸러내는 함수
def preprocessing(data, n):
    min_id = data['아이디'].value_counts() >= n
    min_id = min_id[min_id].index.to_list()
    data = data[data['아이디'].isin(min_id)]

    min_beer = data['맥주'].value_counts() >= n
    min_beer = min_beer[min_beer].index.to_list()
    data = data[data['맥주'].isin(min_beer)]

    return data

tmp = pd.read_csv('전처리후데이터.csv', encoding='utf-8')

temp=tmp.copy()

# 10번 반복합니다.
for i in range(1,10):
    temp = preprocessing(temp, 10)
    print(temp.shape)

temp.to_csv('정제된데이터.csv', encoding='utf-8')

(37656, 10)
(37647, 10)
(37647, 10)
(37647, 10)
(37647, 10)
(37647, 10)
(37647, 10)
(37647, 10)
(37647, 10)


In [6]:
data = pd.read_csv('정제된데이터.csv', encoding='utf-8', index_col=0)

ratings = data.copy()

# 피벗 테이블을 이용해 유저-아이디 매트릭스 구성
ratings_matrix = ratings.pivot_table('평점', index='아이디', columns='맥주')
ratings_matrix.head(3)
# fillna함수를 이용해 Nan처리
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

맥주,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Brooklyn Special Effects (Hoppy Amber/Lager),Budweiser,Carlsberg Pilsner,...,Stephans Bräu Hefe-Weizen Naturtrüb Alkoholfrei,Stephans Bräu Premium Lager,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.0,0.0,0.0,0.0,2.6,0.0,0.0,0.0,2.0,1.8,...,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,4.1
"00cobraR(1,103)",0.0,0.0,0.0,0.0,2.3,0.0,0.0,0.0,1.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0
12HundredBaud(262),2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16erBlech(560),0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.5
"17Beer(1,772)",3.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2,...,0.0,0.0,0.0,0.0,2.6,0.0,2.8,0.0,0.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"zenkis(1,080)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ziobarba2000(122),0.0,0.0,0.0,1.6,1.6,0.0,0.0,0.0,2.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4
zombywoof(541),0.0,0.0,0.0,1.5,1.8,0.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5


In [7]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T

아이디,007lund(92),"00cobraR(1,103)",12HundredBaud(262),16erBlech(560),"17Beer(1,772)","17thfloor(2,444)","17wojownik(1,481)",184601(206),"1FastSTi(3,361)",20107589(748),...,zappafan99(463),zborgerd(187),zdet444dd(440),"zdk(1,917)","zebracakes(1,339)","zenkis(1,080)",ziobarba2000(122),zombywoof(541),"zvikar(12,883)","zvsn(8,580)"
맥주,,,,,,,,,,,,,,,,,,,,,
Asahi Super Dry,0.0,0.0,2.3,0.0,3.4,2.0,0.0,2.4,0.0,0.0,...,0.0,1.8,1.8,1.8,2.7,0.0,0.0,0.0,1.2,0.0
Asahi Super Dry Black,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0
Bavaria 8.6 (Original),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Bavaria Pilsener / Premium Beer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,1.6,1.5,0.0,0.0
Beck's,2.6,2.3,0.0,0.0,0.0,2.8,2.9,2.8,2.5,2.1,...,2.3,0.0,0.0,1.8,2.1,0.0,1.6,1.8,2.1,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tiger Radler Lemon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tsingtao,2.5,2.1,0.0,3.0,2.8,2.5,2.1,0.9,0.0,0.0,...,0.0,2.3,2.0,1.8,1.7,0.0,0.0,0.0,1.7,0.0
Tsingtao Wheat Beer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# 아이템-유저 매트릭스로부터 코사인 유사도 구하기
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity()로 반환된 넘파이 행렬에 영화명을 매핑해 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(62, 62)


맥주,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Brooklyn Special Effects (Hoppy Amber/Lager),Budweiser,Carlsberg Pilsner,...,Stephans Bräu Hefe-Weizen Naturtrüb Alkoholfrei,Stephans Bräu Premium Lager,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
맥주,,,,,,,,,,,,,,,,,,,,,
Asahi Super Dry,1.000000,0.192838,0.303074,0.477367,0.594786,0.256563,0.370012,0.153846,0.601733,0.585606,...,0.073613,0.118367,0.154830,0.243477,0.603275,0.077462,0.673088,0.094640,0.040455,0.587486
Asahi Super Dry Black,0.192838,1.000000,0.129593,0.163273,0.176342,0.113860,0.205111,0.097148,0.174140,0.152539,...,0.057185,0.071062,0.061013,0.375159,0.202128,0.151326,0.139938,0.086948,0.063515,0.179283
Bavaria 8.6 (Original),0.303074,0.129593,1.000000,0.407480,0.343362,0.243339,0.307179,0.178377,0.329092,0.344865,...,0.154946,0.184578,0.176033,0.114390,0.335080,0.079040,0.278518,0.121928,0.082066,0.340135


In [9]:
ratings_matrix.columns

Index(['Asahi Super Dry', 'Asahi Super Dry Black', 'Bavaria 8.6 (Original)',
       'Bavaria Pilsener / Premium Beer', 'Beck's',
       'Berliner Kindl Jubiläums Pilsener Premium', 'Bintang Pilsener',
       'Brooklyn Special Effects (Hoppy Amber/Lager)', 'Budweiser',
       'Carlsberg Pilsner', 'Cass Fresh', 'Desperados',
       'Edelweiss Weissbier Snowfresh', 'Egger Grapefruit Radler',
       'Egger Märzenbier', 'Egger Zwickl', 'Eichbaum Apostel Bräu',
       'Erdinger Weissbier', 'Erdinger Weissbier Dunkel',
       'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
       'Gambrinus Originál 10° (Desítka)', 'Grolsch Premium Lager',
       'Guinness Draught', 'Guinness Original 4.2% (Ireland/UK)',
       'Harbin Beer', 'Heineken', 'Heineken Dark Lager', 'Hoegaarden Rosée',
       'Holsten Premium Bier', 'Jeju Wit Ale', 'Kirin Ichiban',
       'Kloud Fitz Super Clear', 'Kloud Original Gravity',
       'Kozel Černý (Dark) 10°', 'Krombacher Weizen', 'Kronenbourg 1664 Blanc',
       '

In [10]:
len(ratings_matrix.columns)

62

In [14]:
item_sim_df['Cass Fresh'].sort_values(ascending=False)[:6]

맥주
Cass Fresh                1.000000
Kloud Original Gravity    0.338109
Asahi Super Dry           0.304020
Bintang Pilsener          0.303534
Tsingtao                  0.298367
Tiger Beer                0.292789
Name: Cass Fresh, dtype: float64

In [15]:
# ratings_arr.dot(item_sim_arr)는 평점 * 맥주 유사도
# ratings_arr는 사용자 u의 아이템 i와 가장 유사도가 높은 Top_N개 아이템에 대한 실제 평점 벡터
# item_sim_arr는 아이템 i와 가장 유사도가 높은 Top_N개 아이템의 유사도 벡터
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

# 개인화된 예측 평점 구하기
# 평점 value와 유사도 value만 뽑아서 대입
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns = ratings_matrix.columns)

# 개인별로 계산된 예측 평점
ratings_pred_matrix

맥주,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Brooklyn Special Effects (Hoppy Amber/Lager),Budweiser,Carlsberg Pilsner,...,Stephans Bräu Hefe-Weizen Naturtrüb Alkoholfrei,Stephans Bräu Premium Lager,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.830247,0.464353,0.609566,0.721264,0.882396,0.536731,0.623911,0.526778,0.894330,0.854278,...,0.344027,0.377263,0.437888,0.533208,0.768493,0.333235,0.903662,0.352088,0.319651,0.898595
"00cobraR(1,103)",0.747413,0.426077,0.549200,0.659456,0.800791,0.504170,0.569163,0.502076,0.802821,0.746251,...,0.338513,0.345170,0.407167,0.480832,0.695786,0.310733,0.813944,0.336061,0.304393,0.777859
12HundredBaud(262),1.047469,0.716956,0.898372,0.981373,1.058618,0.850851,0.914252,0.896315,1.054977,1.034728,...,0.601529,0.578015,0.693760,0.769033,1.009526,0.561211,1.045627,0.589380,0.508457,1.065502
16erBlech(560),0.977967,0.716978,0.880086,0.935703,1.002051,1.035215,0.902214,0.917737,0.996266,0.982157,...,0.684407,0.689224,0.799116,0.782026,0.978688,0.505414,1.032989,0.648536,0.670105,1.044430
"17Beer(1,772)",1.628774,1.016329,1.213915,1.382141,1.558290,1.113836,1.253211,1.149546,1.578186,1.584507,...,0.784144,0.798148,0.914180,1.086874,1.518930,0.773507,1.652778,0.740375,0.702984,1.599425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"zenkis(1,080)",0.660230,0.474886,0.606485,0.643625,0.690223,0.630128,0.617626,0.939169,0.691218,0.678624,...,0.427759,0.479138,0.551914,0.493900,0.666493,0.326297,0.672037,0.471436,0.523097,0.691493
ziobarba2000(122),0.543884,0.375741,0.482153,0.555188,0.586602,0.441893,0.482357,0.449097,0.608780,0.553531,...,0.304178,0.296713,0.365738,0.410117,0.536598,0.252975,0.561619,0.297174,0.266147,0.600395
zombywoof(541),0.958309,0.596603,0.761325,0.915306,1.034196,0.709580,0.773504,0.723623,1.028645,1.016272,...,0.486334,0.484786,0.569670,0.649283,0.921549,0.459896,0.998111,0.464706,0.442033,1.040506


In [16]:
# 우리가 예측한 평점과 실제 평점간의 차이를 MSE로 계산
def get_mse(pred, actual):
    # 평점이 있는 실제 영화만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 MSE: ', 
      get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE:  2.4824735020920556


In [17]:
# 3개의 col까지만. 3개의 맥주에 대해서 유사도가 큰 5개 선택
top_n_items = [np.argsort(item_sim_df.values[:,3])[:-5:-1]]
top_n_items

[array([ 3, 21, 25,  4], dtype=int64)]

In [18]:
# 따라서 가장 비슷한 유사도를 가지는 맥주만 유사도 벡터로 사용
# 특정 맥주와 비슷한 유사도를 가지는 맥주 Top_N에 대해서만 적용 -> 시간오래걸림

def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 맥주 개수만큼 루프
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개의 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점 계산 : 각 col 맥주별(1개), 2496 사용자들의 예측평점
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col,:][top_n_items].dot(
            ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum(item_sim_arr[col,:][top_n_items])

    return pred

ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=10)
print('아이템 기반 최근접 TOP-N 이웃 MSE: ', 
      get_mse(ratings_pred, ratings_matrix.values))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns=ratings_matrix.columns)

ratings_pred_matrix

아이템 기반 최근접 TOP-N 이웃 MSE:  0.8415119722140935


맥주,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Brooklyn Special Effects (Hoppy Amber/Lager),Budweiser,Carlsberg Pilsner,...,Stephans Bräu Hefe-Weizen Naturtrüb Alkoholfrei,Stephans Bräu Premium Lager,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
007lund(92),1.648999,0.000000,0.144551,0.806088,2.088959,0.000000,0.395821,0.000000,2.254834,2.214076,...,0.000000,0.000000,0.000000,0.000000,1.095597,0.000000,2.143978,0.000000,0.000000,2.147949
"00cobraR(1,103)",1.804934,0.000000,0.244299,1.072148,2.190900,0.000000,0.404648,0.294347,2.237428,1.720944,...,0.000000,0.000000,0.000000,0.000000,1.470760,0.000000,2.186827,0.000000,0.000000,2.118834
12HundredBaud(262),2.008073,0.485180,1.137881,1.902347,2.020672,0.866643,1.376502,1.194773,1.693082,1.583687,...,0.000000,0.000000,0.000000,0.648750,2.384950,0.000000,1.912332,0.219086,0.000000,2.031139
16erBlech(560),1.503961,0.290829,0.716972,1.225656,1.477514,1.944122,0.991361,1.172863,1.269350,1.126509,...,0.000000,0.123406,0.510193,0.402447,1.425351,0.000000,1.603618,0.406801,0.292285,1.591360
"17Beer(1,772)",2.871906,0.623081,1.344698,2.045816,2.243860,0.447018,1.324229,0.918366,2.216382,2.392666,...,0.140844,0.000000,0.000000,0.647291,2.535762,0.152496,3.243602,0.000000,0.000000,2.374642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"zenkis(1,080)",0.858493,0.392181,0.864067,0.953389,1.051963,0.711543,0.721917,1.929184,1.125268,1.028175,...,0.124336,0.209790,0.193880,0.000000,1.406693,0.000000,1.043487,0.294874,0.238852,1.047607
ziobarba2000(122),0.503930,0.000000,0.344708,0.645406,1.180417,0.232515,0.333400,0.183410,1.428321,1.098520,...,0.000000,0.000000,0.174613,0.200649,0.650047,0.000000,0.728712,0.000000,0.000000,1.220632
zombywoof(541),1.850709,0.125395,0.652352,1.881051,2.709271,0.247899,0.736761,0.309839,2.461678,2.516749,...,0.084506,0.000000,0.000000,0.316651,1.934983,0.091497,2.054217,0.000000,0.000000,2.749412


In [19]:
# username='snoworsummer(8,581)'
username='007lund(92)'

# 특정 유저 대상으로 맥주추천
user_rating_id = ratings_matrix.loc[username, :]
# 유저가 먹었던 맥주들 출력
user_rating_id[user_rating_id > 0].sort_values(ascending=False)[:10]

맥주
Warsteiner Premium Verum             4.1
Guinness Draught                     3.7
Paulaner Hefe-Weissbier              3.3
Stella Artois                        2.9
Beck's                               2.6
Sapporo Premium Beer / Draft Beer    2.6
Tsingtao                             2.5
Budweiser                            2.0
Heineken                             2.0
Miller Genuine Draft (MGD)           2.0
Name: 007lund(92), dtype: float64

In [20]:
# 사용자가 안 먹어본 맥주를 추천하자.
def get_not_tried_beer(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 맥주 정보를 추출해 Series로 반환
    # 반환된 user_rating은 영화명(title)을 인덱스로 가지는 Series 객체
    user_rating = ratings_matrix.loc[userId, :]

    # user_rating이 0보다 크면 기존에 관란함 영화.
    # 대상 인덱스를 추출해 list 객체로 만듦
    tried = user_rating[user_rating>0].index.tolist()

    # 모든 맥주명을 list 객체로 만듦
    beer_list = ratings_matrix.columns.tolist()

    # list comprehension으로 tried에 해당하는 영화는 beer_list에서 제외
    not_tried = [beer for beer in beer_list if beer not in tried]

    return not_tried

# 예측 평점 DataFrame에서 사용자 id 인덱스와 not_tried로 들어온 맥주명 추출 후
# 가장 예측 평점이 높은 순으로 정렬
def recomm_beer_by_userid(pred_df, userId, not_tried, top_n):
    recomm_beer = pred_df.loc[userId, not_tried].sort_values(ascending=False)[:top_n]
    return recomm_beer

# 유저가 먹지 않은 맥주이름 추출
not_tried = get_not_tried_beer(ratings_matrix, username)
not_tried

['Asahi Super Dry',
 'Asahi Super Dry Black',
 'Bavaria 8.6 (Original)',
 'Bavaria Pilsener / Premium Beer',
 'Berliner Kindl Jubiläums Pilsener Premium',
 'Bintang Pilsener',
 'Brooklyn Special Effects (Hoppy Amber/Lager)',
 'Cass Fresh',
 'Desperados',
 'Edelweiss Weissbier Snowfresh',
 'Egger Grapefruit Radler',
 'Egger Märzenbier',
 'Egger Zwickl',
 'Eichbaum Apostel Bräu',
 'Erdinger Weissbier',
 'Erdinger Weissbier Dunkel',
 'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
 'Gambrinus Originál 10° (Desítka)',
 'Grolsch Premium Lager',
 'Guinness Original 4.2% (Ireland/UK)',
 'Harbin Beer',
 'Heineken Dark Lager',
 'Hoegaarden Rosée',
 'Holsten Premium Bier',
 'Jeju Wit Ale',
 'Kirin Ichiban',
 'Kloud Fitz Super Clear',
 'Kloud Original Gravity',
 'Kozel Černý (Dark) 10°',
 'Krombacher Weizen',
 'Kronenbourg 1664 Blanc',
 'Lapin Kulta IV A',
 'Leffe Brune / Bruin / Brown',
 'Löwenbräu Original (4.2%)',
 'OB Premier Pilsner',
 'Peroni Nastro Azzurro',
 'Pilsner Urquell',
 'Plat

In [21]:
# top_n과 비슷한 맥주만 추천에 사용
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=5)

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)

# 유저가 먹지 않은 맥주이름 추출
not_tried = get_not_tried_beer(ratings_matrix, username)

# 아이템 기반의 최근접 이웃 CF로 맥주 추천
recomm_beer = recomm_beer_by_userid(ratings_pred_matrix, username, not_tried, top_n=3)
recomm_beer = pd.DataFrame(data=recomm_beer.values, index=recomm_beer.index,
                           columns=['예측평점'])
recomm_beer


,예측평점
맥주,
Grolsch Premium Lager,2.135615
Asahi Super Dry,2.024784
Heineken Dark Lager,1.887716
